### 1. Necessary imports

In [ ]:
import sys
sys.path.append("..")

import logging
import os
# import time
# import datetime
import random
import pandas as pd
import numpy as np
import torch
# import torch.nn.functional as F
import torch.nn as nn
# import torch.optim as optim
import csv
# import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from tqdm import tqdm
# from os.path import join, dirname
# from sklearn.model_selection import train_test_split
from imitation_learning.dataset.frame_dataset import Float115Dataset
from models.mlp_original import MLPModel
from utils.solver import Solver
# from gfootball.env.wrappers import Simple115StateWrapper
from utils.solver import Solver
from torch.utils.tensorboard import SummaryWriter
# from imitation_learning.dataset.dict_dataset import Float115Dataset as DictDataset
# import hydra
# import wandb
# from omegaconf import DictConfig
# import torchtoolbox.transform as transforms
# import torchvision
# import pickle

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [ ]:
# Setting the seeds for result reproducibility
os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
# Loading the dataset containing names of all the frames
logging.info("Reading the dataset")
dataset = pd.read_csv('../data/frames.csv', header=None)[0]
logging.info("Dataset loaded inot the memory")
# Creating Train, Val and Test Dataset
train, val, test = np.split(dataset.sample(frac=1, random_state=42), [
                                    int(.6 * len(dataset)), int(.8 * len(dataset))])

train_frames, val_frames, test_frames = np.array(train, dtype='str'),\
                                        np.array(val, dtype='str'),\
                                        np.array(test, dtype='str')

dataset_path = '/home/ssk/Study/GRP/dataset/npy_files'
train_dataset, val_dataset, test_dataset = Float115Dataset(train_frames, dataset_path), \
                                            Float115Dataset(val_frames, dataset_path), \
                                            Float115Dataset(test_frames, dataset_path)

logging.info(f"Number of training samples: {len(train_dataset)}")
logging.info(f"Number of validation samples: {len(val_dataset)}")
logging.info(f"Number of test samples: {len(test_dataset)}")

In [ ]:
del(train)
del(val)
del(test)
del(dataset)
del(train_frames)
del(val_frames)
del(test_frames)

In [ ]:
batch_size = 128
lr = 0.0106

In [ ]:
# Creating the dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, num_workers=8)


In [ ]:
# Loading the model and defining different parameters for the training
model = MLPModel(hidden_size=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=1, verbose=True, factor=0.2)
criterion = nn.CrossEntropyLoss()

In [ ]:
writer = SummaryWriter()
solver = Solver(model, train_loader, val_loader, criterion, lr, optimizer, writer=writer)
solver.train(epochs=100)
writer.close()

### Save batched data of 128 files

In [ ]:
# A = '007'
# B = '1.000'
# C = '4876'
# print(A.zfill(5))
# print(B.zfill(6))
# print(C.zfill(3))

In [ ]:
from tqdm import tqdm
import pickle
data_save_dir = '/home/ssk/Study/GRP/dataset/batched_data/test'
batch_name_prefix = 'test_batch_'
for i, batch in enumerate(tqdm(val_loader)):
    batch_name = batch_name_prefix + f"{i+1}".zfill(5)
    data_dict = {}
    X, y = batch
    data_dict['obs'] = np.array(X, dtype='float32')
    data_dict['action'] = np.array(y, dtype='float32')
    pkl_file_to_save = os.path.join(data_save_dir, f"{batch_name}.p")
    with open(pkl_file_to_save, 'wb') as handle:
        pickle.dump(data_dict, handle)
    

In [ ]:
with open(os.path.join(data_save_dir, 'train_batch_00001.p'), 'rb') as handle:
    data_dict = pickle.load(handle)
data_dict.keys(), data_dict['obs'].shape, data_dict['action'].shape

### Additional Tasks

In [ ]:
### !!!!!!!!!!!!!!!!!!! Do not delete this cell !!!!!!!!!!!!!!!!!!!
import os
from tqdm import tqdm
import pickle
import time
import numpy as np
from gfootball.env.wrappers import Simple115StateWrapper
import sys
# Here we will write the pickle files
def prepare_npy_dataset_from_replay_files(replay_files, replay_files_path):
    obs_save_dir = '/home/ssk/Study/GRP/dataset/top_480.csv'
#     replay_files_path = '../data/replay_files'

#     if not os.path.exists(obs_save_dir):
#         os.mkdir(obs_save_dir)
    replay_dict = {}
    for replay in tqdm(replay_files):
        with open(os.path.join(replay_files_path, replay), 'rb') as pkl_file:
            episode_data = pickle.load(pkl_file)

        episode_no = replay.split('.')[0]
        episode = episode_data['observations']
        episode['active'] = episode_data['players'][0]['active']
        episode_length = 3002
        raw_obs = {}

        episode_dir = os.path.join(obs_save_dir, episode_no)
#         if not os.path.exists(episode_dir):
#             os.mkdir(episode_dir)

        for step in range(episode_length):
            for (key, item) in episode.items():
                raw_obs[key] = item[step]

            float115_frame =  Simple115StateWrapper.convert_observation([raw_obs], True)[0].tolist()
            action = episode_data['players'][0]['action'][step]
            
            frame_name = episode_no+f'_{step}'
            if len(action) != 0:
                float115_frame.extend(action)
#                 fram_save_path = os.path.join(episode_dir, frame_name)
                replay_dict[frame_name] = np.array(float115_frame, dtype='float32')
                del(float115_frame)
#                 print(f"Size of the frame list : {sys.getsizeof(float115_frame)} bytes")
#                 print(f"Size of the numpy array for the same list : {np.array(float115_frame, dtype='float32').nbytes} bytes")
#                 np.save(fram_save_path, np.array(float115_frame))
        del(episode_data)
        del(episode)
        del(raw_obs)
    
    a_file = open("/home/ssk/Study/GRP/dataset/replay_data_part1.pkl", "wb")
    pickle.dump(replay_dict, a_file)
    a_file.close()
    print(f"Size of the replay dict for one file: {sys.getsizeof(replay_dict)/1024/1024} MB")
    print(len(replay_dict.keys()))

In [ ]:

replay_files_path = '../data/replay_files'
replay_files = sorted(os.listdir(replay_files_path))
replay_files.pop(0)
# replay_files = replay_files[0:1]
print(f"total replay files: {len(replay_files)}")
# replay_files = replay_files[0:1]

start = time.perf_counter()
prepare_npy_dataset_from_replay_files(replay_files[0:2400], replay_files_path)
end =  time.perf_counter()

print(f"Total time needed to process {len(replay_files)}: {end-start}s")
print(f"Time needed to process a single file: {(end-start)/len(replay_files)}s")